In [1]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.signal as ss

def voltage_criterion(data=None, v_thresh=-40, time_thresh=260):
    threshold_crossings = np.diff(data > v_thresh, prepend=False)
    upward_crossings = np.argwhere(threshold_crossings)[::2,0]
    downward_crossings = np.argwhere(threshold_crossings)[1::2,0]
    # If length of threshold_crossings is not even
    if np.mod(np.argwhere(threshold_crossings).reshape(-1,).shape[0],2)!=0:
        legit_up_crossings = upward_crossings[:-1][np.diff(np.argwhere(threshold_crossings).reshape(-1,))[::2]>time_thresh]
        legit_down_crossings = downward_crossings[np.diff(np.argwhere(threshold_crossings).reshape(-1,))[::2]>time_thresh]
    else:
        legit_up_crossings = upward_crossings[np.diff(np.argwhere(threshold_crossings).reshape(-1,))[::2]>time_thresh]
        legit_down_crossings = downward_crossings[np.diff(np.argwhere(threshold_crossings).reshape(-1,))[::2]>time_thresh]
    return upward_crossings, legit_up_crossings, legit_down_crossings

# Input: upward and downward crossings
# Output: bounds of NMDA spikes meeting current criterion
def current_criterion(legit_uc_iso=[], legit_dc_iso=[], p=1, control_inmda=np.array([1])):
    bounds = []
    sum_current = []
    

    for ind1 in np.arange(0,len(legit_uc_iso)):
        e1 = control_inmda[legit_uc_iso[ind1], p] #current @ up_crossing[ind1]
        #all the indices where current crosses 130% of e1
        x30 = np.argwhere(np.diff(control_inmda[legit_uc_iso[ind1]:legit_dc_iso[ind1], p] < 1.3*e1, prepend=False))
        #all the indices where current crosses 115% of e1
        x15 = np.argwhere(np.diff(control_inmda[legit_uc_iso[ind1]:legit_dc_iso[ind1], p] < 1.15*e1, prepend=False))
        
        if len(x30)>0:
        
            x15_copy = x15
            x30_copy = x30
            
            try:
                i = x30[0][0]
            except:
                import pdb; pdb.set_trace()
                
            n = 0
            
            
            while n==0:
                if len(np.sort(x15[x15>i]))!=0:
                    b1 = i
                    b2 = np.sort(x15[x15>i])[0]
                    bounds.append([legit_uc_iso[ind1]+b1,legit_uc_iso[ind1]+b2])
                    sum_current.append(np.sum(control_inmda[legit_uc_iso[ind1]+b1:legit_uc_iso[ind1]+b2,p])/10)
                else:
                    b1 = i
                    b2 = (legit_dc_iso[ind1]-legit_uc_iso[ind1])
                    bounds.append([legit_uc_iso[ind1]+b1,legit_uc_iso[ind1]+b2])
                    sum_current.append(np.sum(control_inmda[legit_uc_iso[ind1]+b1:legit_uc_iso[ind1]+b2,p])/10)
                    n=1
                
                x30_copy = x30_copy[x30_copy>legit_uc_iso[ind1]+b2]
                #import pdb; pdb.set_trace()
                
                if len(x30_copy)!=0:
                    i = x30_copy[x30_copy>b2][0]
                else:
                    n=1
    return bounds, sum_current

In [2]:
# output_folder = 'output_control'
%cd ../scripts/L5BaselineResults/
conns = pd.read_csv('Connections.csv')

/home/drfrbc/Neural-Modeling/scripts/L5BaselineResults


In [3]:
conns.loc[conns.Type=='dend','Sec ID'] = conns.loc[conns.Type=='dend','Name'].apply(lambda x: int(x.split('dend[')[1].split(']')[0]))
conns.loc[conns.Type=='apic','Sec ID'] = conns.loc[conns.Type=='apic','Name'].apply(lambda x: int(x.split('apic[')[1].split(']')[0]))
conns.loc[conns.Type=='soma','Sec ID'] = conns.loc[conns.Type=='soma','Name'].apply(lambda x: int(x.split('soma[')[1].split(']')[0]))
conns.loc[conns.Type=='axon','Sec ID'] = conns.loc[conns.Type=='axon','Name'].apply(lambda x: int(x.split('axon[')[1].split(']')[0]))

conns['X'] = conns['Name'].apply(lambda x: float(x.split('(')[1].split(')')[0]))

conns.rename(columns={'Distance':'conns_Distance'},inplace=True)

conns['Sec ID'] = conns['Sec ID'].astype(int)
conns['X'] = conns['X'].astype(float)


/tmp/ipykernel_609637/286416071.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  conns.loc[conns.Type=='axon','Sec ID'] = conns.loc[conns.Type=='axon','Name'].apply(lambda x: int(x.split('axon[')[1].split(']')[0]))


In [4]:
conns.head()

,Node ID,conns_Distance,Conductance,Type,Name,Source Population,Release Probability,Sec ID,X
0,0,61.505696,5.006073e-08,dend,L5PCtemplate[0].dend[60](0.264706),exc_stim,0.6,60,0.264706
1,0,112.522057,3.250785e-08,dend,L5PCtemplate[0].dend[28](0.128571),exc_stim,0.6,28,0.128571
2,0,73.494480,5.547277e-08,dend,L5PCtemplate[0].dend[29](0.7),exc_stim,0.6,29,0.700000
3,0,101.530468,1.005930e-07,dend,L5PCtemplate[0].dend[33](0.264706),exc_stim,0.6,33,0.264706
4,0,65.675576,3.045596e-08,dend,L5PCtemplate[0].dend[48](0.409091),exc_stim,0.6,48,0.409091


In [5]:
segs = pd.read_csv('Segments.csv')
segs_degrees = pd.read_csv('SegmentsDegrees.csv').groupby(['Type','Sec ID'])['Degrees'].max().reset_index()
segs['segmentID'] = segs.index
segs = segs.set_index(['Type','Sec ID']).join(segs_degrees.set_index(['Type','Sec ID'])).reset_index()

segs['Sec ID'] = segs['Sec ID'].astype(int)
segs['X'] = segs['X'].astype(float)
segs['Elec_distanceQ'] = 'None'

segs.loc[segs.Type=='dend','Elec_distanceQ'] = pd.qcut(segs.loc[segs.Type=='dend','Elec_distance'], 10, labels=False)
segs.loc[segs.Type=='apic','Elec_distanceQ'] = pd.qcut(segs.loc[segs.Type=='apic','Elec_distance'], 10, labels=False)

In [6]:
v = h5py.File('v_report.h5','r')
hva = h5py.File('Ca_HVA.ica_report.h5','r')
lva = h5py.File('Ca_LVAst.ica_report.h5','r')
ih = h5py.File('Ih.ihcn_report.h5','r')

nmda = h5py.File('inmda_report.h5','r')

na = h5py.File('NaTa_t.gNaTa_t_report.h5','r')
spks = h5py.File('spikes.h5','r')

In [7]:
spktimes = spks['spikes']['biophysical']['timestamps'][:]
spkinds = np.sort((spktimes*10).astype(int))

In [8]:
na_df = pd.DataFrame(columns=['segmentID','na_lower_bound'])

for p in segs[(segs.Type=='dend')|(segs.Type=='apic')].index:
    # print(p)
    threshold_crossings = np.diff(na['report']['biophysical']['data'][:,p] > 0.003, prepend=False)
    upward_crossings = np.argwhere(threshold_crossings)[::2,0]
    downward_crossings = np.argwhere(threshold_crossings)[1::2,0]
    # Only count if not within 2 ms after a somatic spike
    na_spks = [i for i in upward_crossings if ~np.any((i-spkinds>=-5) & (i-spkinds<50))]
    
    # if len(na_spks)>0:
    #     na_df = na_df.append(pd.DataFrame({'segmentID':np.tile(p,len(na_spks)),
    #                                            'na_lower_bound':na_spks}))
    # else:
    #     na_df = na_df.append(pd.DataFrame({'segmentID':p,
    #                                            'na_lower_bound':np.array([np.nan])}))

    if len(na_spks) > 0:
        new_rows = pd.DataFrame({'segmentID': np.tile(p, len(na_spks)),
                                 'na_lower_bound': na_spks})
    else:
        new_rows = pd.DataFrame({'segmentID': [p],
                                 'na_lower_bound': [np.nan]})
    
    na_df = pd.concat([na_df, new_rows], ignore_index=True)

In [9]:

na_df.reset_index(inplace=True)

In [10]:
def reshape_pad(v, fillval=np.nan):
    lens = np.array([len(item) for item in v])
    mask = lens[:,None] > np.arange(lens.max())
    out = np.full(mask.shape,fillval)
    out[mask] = np.concatenate(v)
    return out

In [11]:
# get the value of the conductance at the peak
df = pd.DataFrame(columns=['segmentID','na_lower_bound','peak_value'])

for s in na_df[~pd.isnull(na_df.na_lower_bound)].segmentID.unique():

    inds = na_df[na_df.segmentID==s]['na_lower_bound'].astype(int).values
    
    L = np.split(na['report']['biophysical']['data'][:,s], inds)
    
    X = reshape_pad(L)
    
    peak_vals = np.max(X[:,0:10], axis=1)
    
    df = pd.concat((df,pd.DataFrame({'segmentID': s*np.ones((inds.shape[0],)).astype(int),
                                     'na_lower_bound': inds, 
                                     'peak_value': peak_vals[1:]})))
    
    # print(s)

/tmp/ipykernel_609637/1715638600.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat((df,pd.DataFrame({'segmentID': s*np.ones((inds.shape[0],)).astype(int),


In [12]:
na_df.to_csv('na_df.csv',index=False)

In [13]:
nmda_df = pd.DataFrame(columns=['segmentID','nmda_lower_bound', 'nmda_upper_bound', 'mag'])
for p in segs[(segs.Type=='dend')|(segs.Type=='apic')].index:
    print(p)
    legit_uc = voltage_criterion(data=v['report']['biophysical']['data'][:-10000,p],v_thresh=-40, time_thresh=260)[1]
    legit_dc = voltage_criterion(data=v['report']['biophysical']['data'][:-10000,p],v_thresh=-40, time_thresh=260)[-1]
    
    legit_uc_iso = legit_uc
    
    legit_dc_iso = legit_dc
    
    
    if (len(legit_uc_iso)!=0) & (np.min(nmda['report']['biophysical']['data'][:,p])!=0):
        bnds, sum_curr = current_criterion(legit_uc_iso=legit_uc_iso, 
                                           legit_dc_iso=legit_dc_iso, 
                                           p=p, 
                                           control_inmda=nmda['report']['biophysical']['data'])
        nmda_df = nmda_df.append(pd.DataFrame({'segmentID':np.tile(p,len(bnds)),
                                               'nmda_lower_bound':np.array(bnds).reshape(-1,2)[:,0],
                                               'nmda_upper_bound':np.array(bnds).reshape(-1,2)[:,1],
                                               'mag':sum_curr}))
    
    else:
        nmda_df = nmda_df.append(pd.DataFrame({'segmentID':np.tile(p,1),
                                               'nmda_lower_bound':np.nan,
                                               'nmda_upper_bound':np.nan,
                                               'mag':np.nan}))

0


AttributeError: 'DataFrame' object has no attribute 'append'

NMDA effect on calcium

In [ ]:
def get_nmda_sta_calcium(segs_nmda_df, spktimes):    
    nmda_dend = np.zeros((10,27,))
    nmda_apic = np.zeros((10,27,))
    
    edges_dend=np.quantile(segs_nmda_df[(segs_nmda_df.Type=='dend')]['Elec_distance_nexus'].unique(),np.arange(0,1.1,0.1))
    edges_apic=np.quantile(segs_nmda_df[(segs_nmda_df.Type=='apic')]['Elec_distance_nexus'].unique(),np.arange(0,1.1,0.1))
    
    actual_spktimes = []
    c = 0
    for i in np.sort(spktimes):
        # excludes bursts
        if i-c > 10:
            for e in np.arange(0,10):
                nmda_inds = segs_nmda_df[(segs_nmda_df.mag<-0.1)&
                                     (segs_nmda_df.Type=='dend')&
                                     (segs_nmda_df.Elec_distance_nexus>edges_dend[e])&
                                     (segs_nmda_df.Elec_distance_nexus<=edges_dend[e+1])]['nmda_lower_bound'].values.astype(int)
    
                x2, _ = np.histogram(nmda_inds/10,bins=np.arange(np.floor(i)-100,np.floor(i)+40,5))
                nmda_dend[e] += x2
                
                nmda_inds = segs_nmda_df[(segs_nmda_df.mag<-0.1)&
                                     (segs_nmda_df.Type=='apic')&
                                     (segs_nmda_df.Elec_distance_nexus>edges_apic[e])&
                                     (segs_nmda_df.Elec_distance_nexus<=edges_apic[e+1])]['nmda_lower_bound'].values.astype(int)
    
                x2, _ = np.histogram(nmda_inds/10,bins=np.arange(np.floor(i)-100,np.floor(i)+40,5))
                nmda_apic[e] += x2
            
            actual_spktimes.append(i)
        c = i
        
    return nmda_dend, nmda_apic

In [ ]:
segs_nmda_df = segs.set_index('segmentID').join(nmda_df.set_index('segmentID')).reset_index()

edges_apic=np.quantile(segs_nmda_df[(segs_nmda_df.Type=='apic')]['Elec_distance_nexus'].unique(),np.arange(0,1.1,0.1))

In [ ]:
for i in np.arange(0,len(edges_apic)-1):
    plt.scatter(segs_nmda_df[(segs_nmda_df.Type=='apic')&
                         (segs_nmda_df.Elec_distance_nexus>=edges_apic[i])&
                         (segs_nmda_df.Elec_distance_nexus<edges_apic[i+1])]['Coord X'],
                segs_nmda_df[(segs_nmda_df.Type=='apic')&
                         (segs_nmda_df.Elec_distance_nexus>=edges_apic[i])&
                         (segs_nmda_df.Elec_distance_nexus<edges_apic[i+1])]['Coord Y'])

In [ ]:
ca_spktimes_nodupes = np.sort(ca_spktimes[1:])[np.diff(np.sort(ca_spktimes))>100]

In [ ]:
nmda_dend, nmda_apic = get_nmda_sta_calcium(segs_nmda_df,ca_spktimes_nodupes)
rand_spktimes = np.sort(np.random.choice(np.arange(0,150000), ca_spktimes_nodupes.shape[0]))
nmda_dend_rand, nmda_apic_rand = get_nmda_sta_calcium(segs_nmda_df, rand_spktimes)

In [ ]:
plt.scatter(np.arange(0,df.shape[0]), np.mean(f['report']['biophysical']['data'],axis=0))
plt.xlim(1740,1760)

In [ ]:
exc_syns[exc_syns.Type=='apic']['num_syns'].sum() / exc_syns[exc_syns.Type=='apic']['Section_L'].unique().sum()

In [ ]:
exc_syns[exc_syns.Type=='dend']['num_syns'].sum() / exc_syns[exc_syns.Type=='dend']['Section_L'].unique().sum()

In [ ]:
t = 5325
plt.plot(np.arange(-10,10,.1), (v['report']['biophysical']['data'][t-100:t+100,101]-45)*na['report']['biophysical']['data'][t-100:t+100,101],color='r')
plt.vlines(-7,-0.1,-0.15,color='r')
#plt.hlines(0,-10,10,color='r',linestyle='dashed',alpha=0.2)
plt.text(-6,-0.125,'0.05 mA/cm2')
plt.ylabel('current (mA/cm2)')

plt.twinx()
plt.plot(np.arange(-10,10,.1), v['report']['biophysical']['data'][t-100:t+100,101],color='k')
plt.plot(np.arange(-10,10,.1), v['report']['biophysical']['data'][t-100:t+100,0],color='k',alpha=0.2)
plt.text(5,-70,'soma',alpha=0.2)
plt.vlines(5,-10,-20,color='k')
plt.hlines(-20,5,10,color='k')
#plt.hlines(-60,-10,10,color='k',linestyle='dashed',alpha=0.2)
#plt.text(10,-60,'-60 mV')
plt.ylabel('voltage (mV)')
plt.text(7,-25,'5 ms')
plt.text(2,-15,'10 mV')
plt.title('dendritic Na+ spike')

plt.savefig('exNaspike.svg')

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(2,1,1)
plt.title('NMDA spikes - apical')
x = (nmda_apic-nmda_apic_rand)/ca_spktimes_nodupes.shape[0]
plt.imshow(x,cmap='jet',vmin = -1, vmax = 5)
plt.xticks(ticks=np.arange(0,26,4)+0.5,labels=['{}'.format(i) for i in np.arange(-100,40,20)])
plt.yticks(ticks=[0,9],labels=['further','closer'])
plt.colorbar(label='additional events per Ca2+ spike')
plt.xlabel('time (ms)')
plt.xlim(5,26)

plt.subplot(2,1,2)
plt.title('NMDA spikes - basal')
x = (nmda_dend-nmda_dend_rand)/ca_spktimes_nodupes.shape[0]
plt.imshow(x,cmap='jet',vmin = -1, vmax = 5)
plt.xticks(ticks=np.arange(0,26,4)+0.5,labels=['{}'.format(i) for i in np.arange(-100,40,20)])
plt.yticks(ticks=[0,9],labels=['further','closer'])
plt.colorbar(label='additional events per Ca2+ spike')
plt.xlim(5,26)

plt.xlabel('time (ms)')
plt.savefig('nmdaspikes.svg')
plt.show()

In [ ]:
#for i in na_df.segmentID.unique():
#    seg = na_df.loc[i,'segmentID']
#    if not pd.isnull(na_df.loc[i,'na_lower_bound']):
#        spkt = int(na_df.loc[i,'na_lower_bound'])
#        trace = na['report']['biophysical']['data'][spkt-10:spkt+10,seg]
#        peak_value = np.max(trace)
#        half_peak = peak_value/2
#        duration = np.arange(0,20)[trace>half_peak] + spkt - 10
#        na_df.loc[i,'duration_low'] = duration[0]
#        na_df.loc[i,'duration_high'] = duration[-1]
#        na_df.loc[i,'peak_value'] = peak_value
#    else:
#        na_df.loc[i,'duration_low'] = np.nan
#        na_df.loc[i,'duration_high'] = np.nan
#        na_df.loc[i,'peak_value'] = np.nan
#        
#na_df['duration'] = (na_df['duration_high'] - na_df['duration_low'] + 1)/10

In [ ]:
nmda_df = pd.read_csv('nmda_df.csv')

In [ ]:
nmda_df['duration'] = nmda_df['nmda_upper_bound'] - nmda_df['nmda_lower_bound']

In [ ]:
p = np.random.choice(nmda_df[(~pd.isnull(nmda_df.nmda_lower_bound))&(nmda_df.duration<300)].index)
nmda_df.loc[p,'segmentID'], nmda_df.loc[p,'nmda_lower_bound']

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(2,1,1)
plt.title('NMDA spikes - apical')
plt.imshow((nmda_apic-nmda_apic_rand)/len(spktimes),cmap='jet',vmin=-0.7,vmax=0.7)
plt.xticks(ticks=np.arange(0,26,4),labels=['{}'.format(i) for i in np.arange(-100,40,20)])
plt.yticks(ticks=[0,9],labels=['further','closer'])
plt.colorbar(label='additional events per AP')
plt.xlabel('time (ms)')
plt.xlim(5,26)

plt.subplot(2,1,2)
plt.title('NMDA spikes - basal')
plt.imshow((nmda_dend-nmda_dend_rand)/len(spktimes),cmap='jet', vmin=-0.7, vmax=0.7)
plt.xticks(ticks=np.arange(0,26,4),labels=['{}'.format(i) for i in np.arange(-100,40,20)])
plt.yticks(ticks=[0,9],labels=['further','closer'])
plt.colorbar(label='additional events per AP')
plt.xlim(5,26)

plt.xlabel('time (ms)')
plt.savefig('nmdaspikes.svg')
plt.show()

In [ ]:
plt.hist(control_v[10000,:])

In [ ]:
control_v[10000,0]

In [ ]:
seg = 1842
lower_bound = 15000
plt.figure(figsize=(12,10))
plt.plot(np.arange(0,600,.1),-(hva['report']['biophysical']['data'][lower_bound-1000:lower_bound+5000,seg]+\
         lva['report']['biophysical']['data'][lower_bound-1000:lower_bound+5000,seg]+\
         ih['report']['biophysical']['data'][lower_bound-1000:lower_bound+5000,seg]),color='b')
plt.hlines(np.mean(-(hva['report']['biophysical']['data'][lower_bound-1000:lower_bound+5000,seg]+\
         lva['report']['biophysical']['data'][lower_bound-1000:lower_bound+5000,seg]+\
         ih['report']['biophysical']['data'][lower_bound-1000:lower_bound+5000,seg])),0,600,color='b')
plt.xlim(0,200)

plt.twinx()
plt.plot(np.arange(0,600,.1),v['report']['biophysical']['data'][lower_bound-1000:lower_bound+5000,seg],color='k')
plt.plot(np.arange(0,600,.1),v['report']['biophysical']['data'][lower_bound-1000:lower_bound+5000,0],color='k',alpha=0.2)
plt.xlim(0,200)

In [ ]:
ca_df = pd.DataFrame(columns=['segmentID','ca_lower_bound'])
#&(segs['Coord Y']>630) & (segs['Coord Y']<830)
segIDs = segs[(segs.Type=='apic')&(segs['Coord Y']>400)&(segs['Coord Y']<1000)]['segmentID']
for p in segIDs:
    print(p)
    trace = hva['report']['biophysical']['data'][:,p]+\
                lva['report']['biophysical']['data'][:,p]+\
                ih['report']['biophysical']['data'][:,p]
    m = np.mean(trace)
    s = np.std(trace)

    legit_uc = voltage_criterion(data=v['report']['biophysical']['data'][:-10000,p],v_thresh=-40, time_thresh=200)[1]
    legit_dc = voltage_criterion(data=v['report']['biophysical']['data'][:-10000,p],v_thresh=-40, time_thresh=200)[-1]
    
    legit_uc_iso = legit_uc
    
    legit_dc_iso = legit_dc
    
    
    if (len(legit_uc_iso)!=0) & (np.min(trace)!=0):
        bnds, sum_curr = current_criterion(legit_uc_iso=legit_uc_iso, 
                                           legit_dc_iso=legit_dc_iso, 
                                           p=p, 
                                           control_inmda=hva['report']['biophysical']['data'])
        ca_df = ca_df.append(pd.DataFrame({'segmentID':np.tile(p,len(bnds)),
                                               'ca_lower_bound':np.array(bnds).reshape(-1,2)[:,0],
                                               'ca_upper_bound':np.array(bnds).reshape(-1,2)[:,1],
                                               'mag':sum_curr}))
    
    else:
        ca_df = ca_df.append(pd.DataFrame({'segmentID':np.tile(p,1),
                                               'ca_lower_bound':np.nan,
                                               'ca_upper_bound':np.nan,
                                               'mag':np.nan}))

In [ ]:
segs_ca_df = segs.set_index('segmentID').join(ca_df.set_index('segmentID')).reset_index() 

ca_spktimes = segs_ca_df[(segs_ca_df.Type=='apic') & (segs_ca_df['Sec ID']==50)]['ca_lower_bound']/10
#ca_spktimes = segs_ca_df[segs_ca_df.segmentID==1738]['ca_lower_bound']/10

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(2,1,1)
plt.title('NMDA spikes - apical')
plt.imshow((nmda_apic-nmda_apic_rand)/len(spktimes),cmap='jet')
plt.xticks(ticks=np.arange(0,26,4),labels=['{}'.format(i) for i in np.arange(-100,40,20)])
plt.yticks(ticks=[0,9],labels=['further','closer'])
plt.colorbar(label='additional events per AP')
plt.xlabel('time (ms)')
plt.xlim(5,26)

plt.subplot(2,1,2)
plt.title('NMDA spikes - basal')
plt.imshow((nmda_dend-nmda_dend_rand)/len(spktimes),cmap='jet')
plt.xticks(ticks=np.arange(0,26,4),labels=['{}'.format(i) for i in np.arange(-100,40,20)])
plt.yticks(ticks=[0,9],labels=['further','closer'])
plt.colorbar(label='additional events per AP')
plt.xlim(5,26)

plt.xlabel('time (ms)')
plt.savefig('nmdaspikes.svg')
plt.show()

In [ ]:
nmda_spktimes = nmda_df[nmda_df.mag<-0.1]['nmda_lower_bound']/10
n = np.zeros((12,))
for i in spktimes:
    x,y = np.histogram(nmda_spktimes[(nmda_spktimes>i-30) & (nmda_spktimes<i+30)]-i,bins=np.arange(-30,31,5))
    n+=x

In [ ]:
plt.plot(np.arange(-30,30,5),n)

In [ ]:
t = 643309
s = 76

plt.plot(np.arange(-20,50,.1),nmda['report']['biophysical']['data'][t-200:t+500,s],color='r')
plt.vlines(40,-0.005,-0.01,color='r')
plt.text(42,-0.007,'5 pA')
plt.ylabel('current (nA)')

plt.twinx()

plt.plot(np.arange(-20,50,.1),v['report']['biophysical']['data'][t-200:t+500,s],color='k')
plt.plot(np.arange(-20,50,.1),v['report']['biophysical']['data'][t-200:t+500,0],color='k',alpha=0.2)
plt.text(45,-60,'soma',alpha=0.2)
plt.hlines(-40,-15,-5)
plt.vlines(-15,-30,-40)
plt.text(-23,-35,'10 mV')
plt.text(-12,-45,'10 ms')
plt.ylabel('voltage (mV)')
plt.title('NMDA spike')
plt.savefig('NMDAspike.svg')

In [ ]:
exc_syns.loc[(pd.isnull(exc_syns.num_syns))|(exc_syns.num_syns>30),'num_syns'] = 0
inh_syns.loc[pd.isnull(inh_syns.num_syns),'num_syns'] = 0

In [ ]:
p = 829#np.random.choice(segs[segs.Type=='dend'].index)
plt.title('dend {}'.format(p))
plt.plot(control_v[7200:7700,p],label='dend')
plt.plot(control_v[7200:7700,0],label='soma')
plt.hlines(-40,0,500)
plt.twinx()
plt.plot(control_inmda[7200:7700,p],color='r')
plt.legend()

In [ ]:
na_df.to_csv('64Hz_soma_inh_na_df.csv')
na_df = pd.read_csv('na_df.csv')

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(np.arange(0,400,.1),nmda['report']['biophysical']['data'][s1:s1+4000,t],color='b',alpha=0.1)
plt.vlines(400,0,0.003,color='b')
plt.text(405,0.002,'0.003')
plt.box(False)
plt.xticks([])
plt.yticks([])
plt.twinx()
plt.plot(np.arange(0,400,.1),100*na['report']['biophysical']['data'][s1:s1+4000,t],color='r',alpha=0.1)
plt.vlines(300,0.09,0.12,color='r')
plt.text(305,0.1,'0.03')
plt.box(False)
plt.xticks([])
plt.yticks([])
plt.savefig('fig2F_na_nmda.svg')

In [ ]:
na_df = pd.read_csv('na_df.csv')
nmda_df = pd.read_csv('nmda_df.csv')
ca_df = pd.read_csv('ca_df.csv')

In [ ]:
import matplotlib
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)

plt.figure(figsize=(12,10))
plt.plot(np.arange(0,60000,.1),f['report']['biophysical']['data'][:,175],color='b',label='dend voltage')
plt.plot(np.arange(0,60000,.1),f['report']['biophysical']['data'][:,0],color='k',alpha=0.3,label='soma voltage')
plt.legend(loc='lower left')
plt.hlines(-40,0,60000,linestyle='dashed')
plt.ylabel('voltage (mV)')
plt.ylim(-85,25)
plt.xlabel('time (ms)')
plt.twinx()
plt.plot(np.arange(0,60000,.1),h['report']['biophysical']['data'][:,175],color='r',label='inmda dend')
plt.legend(loc='upper right')
plt.plot(np.arange(0,60000,.1)[nmda_df[nmda_df.seg_id==175]['nmda_lower_bound'].values.astype(int)],
         h['report']['biophysical']['data'][nmda_df[nmda_df.seg_id==175]['nmda_lower_bound'].values.astype(int),175],'m*',markersize=20)
plt.plot(np.arange(0,60000,.1)[nmda_df[nmda_df.seg_id==175]['nmda_upper_bound'].values.astype(int)],
         h['report']['biophysical']['data'][nmda_df[nmda_df.seg_id==175]['nmda_upper_bound'].values.astype(int),175],'k*',markersize=20)
plt.xlim(32000,32400)
plt.ylim(-0.009,0.001)
plt.ylabel('current (nA)')

In [ ]:
exc_syns.head()

In [ ]:
plt.plot(exc_syns[exc_syns.Type=='apic']['Sec ID'], exc_syns[exc_syns.Type=='apic']['Section_diam'], '.')

In [ ]:
for i in np.random.choice(na_df[(na_df.na_lower_bound>20) & (na_df.na_lower_bound<1400000)].index,10000):
    seg = na_df.loc[i,'segmentID']
    if not pd.isnull(na_df.loc[i,'na_lower_bound']):
        spkt = int(na_df.loc[i,'na_lower_bound'])
        trace = na['report']['biophysical']['data'][spkt-10:spkt+10,seg]
        peak_value = np.max(trace)
        half_peak = peak_value/2
        duration = np.arange(0,20)[trace>half_peak] + spkt - 10
        na_df.loc[i,'duration_low'] = duration[0]
        na_df.loc[i,'duration_high'] = duration[-1]
        na_df.loc[i,'peak_value'] = peak_value
    else:
        na_df.loc[i,'duration_low'] = np.nan
        na_df.loc[i,'duration_high'] = np.nan
        na_df.loc[i,'peak_value'] = np.nan

In [ ]:
nmda_apic = np.zeros((10,27,))
mean_nmda = np.zeros((1,1400))
nmda_traces = []

nmda_inds = segs_nmda_df[(segs_nmda_df.mag<-0.1)&
                             (segs_nmda_df.Type=='apic')]['nmda_lower_bound'].values.astype(int)

for i in np.random.choice(ca_spktimes[ca_spktimes>100], 100):
    x2, _ = np.histogram(nmda_inds/10,bins=np.arange(np.floor(i)-100,np.floor(i)+40,5))
    nmda_apic[0,:] += x2
    # mean of i_nmda across all apical segments that had an nmda spike in this window
    segs_of_interest = segs_nmda_df[(segs_nmda_df.mag<-0.1)&
                                    (segs_nmda_df['Coord Y']>800)&
                                    (segs_nmda_df['Coord Y']<900)&
                                    (segs_nmda_df.Type=='apic')&
                                    (segs_nmda_df.nmda_lower_bound>int(np.floor(i)-100)*10)&
                                    (segs_nmda_df.nmda_lower_bound<int(np.floor(i)+40)*10)]['segmentID'].values
    nmda_traces.append(nmda['report']['biophysical']['data'][int(np.floor(i)-100)*10:int(np.floor(i)+40)*10,
                                                                    np.unique(np.sort(segs_of_interest))])

In [ ]:
na_df['duration'] = (na_df['duration_high'] - na_df['duration_low'] + 1)/10

In [ ]:
T = []
for i in np.arange(0,len(nmda_traces)):
    #plt.plot(nmda_traces[i][:,0],color='k',alpha=0.2)
    T.append(nmda_traces[i][:,0])
plt.plot(np.mean(np.array(T),axis=0),color='r')

In [ ]:
segs = pd.read_csv('/Volumes/TOSHIBA EXT/L5NeuronSimulation_new/L5NeuronSimulation/MorphAnalysis/Segments.csv')
segs_degrees = pd.read_csv('SegmentsDegrees.csv').groupby(['Type','Sec ID'])['Degrees'].max().reset_index()
segs['segmentID'] = segs.index
segs = segs.set_index(['Type','Sec ID']).join(segs_degrees.set_index(['Type','Sec ID'])).reset_index()

segs['Sec ID'] = segs['Sec ID'].astype(int)
segs['X'] = segs['X'].astype(float)
segs['Elec_distanceQ'] = 'None'

segs.loc[segs.Type=='dend','Elec_distanceQ'] = pd.qcut(segs.loc[segs.Type=='dend','Elec_distance'], 10, labels=False)
segs.loc[segs.Type=='apic','Elec_distanceQ'] = pd.qcut(segs.loc[segs.Type=='apic','Elec_distance'], 10, labels=False)

In [ ]:
df_gb = df.groupby('segmentID')['na_lower_bound'].count().sort_values().reset_index()

In [ ]:
color_field = 'Elec_distance'

ax = plt.figure(figsize=(2,5))
import matplotlib
from matplotlib.colors import Normalize
cmap = matplotlib.cm.get_cmap('jet')
norm = matplotlib.colors.Normalize(vmin = np.log10(exc_syns['Elec_distance'].min()), vmax = np.log10(exc_syns['Elec_distance'].max()))

for i in exc_syns[exc_syns.Type=='apic']['Sec ID'].unique():
    section = exc_syns[(exc_syns.Type=='apic')&(exc_syns['Sec ID']==i)]
    for j in section.index.tolist()[:-1]:
        plt.plot(section.loc[j:j+1,'Coord X'],
                 section.loc[j:j+1,'Coord Y'],
             color=cmap(norm(np.log10(section.loc[j:j+1,color_field].values[0]))),
             linewidth = 1*section.loc[j:j+1,'Section_diam'].unique())
    
for i in exc_syns[exc_syns.Type=='dend']['Sec ID'].unique():
    section = exc_syns[(exc_syns.Type=='dend')&(exc_syns['Sec ID']==i)]
    for j in section.index.tolist()[:-1]:
        plt.plot(section.loc[j:j+1,'Coord X'],
                 section.loc[j:j+1,'Coord Y'],
             color=cmap(norm(np.log10(section.loc[j:j+1,color_field].values[0]))),
             linewidth = 1*section.loc[j:j+1,'Section_diam'].unique())
        
plt.scatter(exc_syns[(exc_syns.Type=='soma')&(exc_syns['Sec ID']==0)].loc[2523,'Coord X'],
         exc_syns[(exc_syns.Type=='soma')&(exc_syns['Sec ID']==0)].loc[2523,'Coord Y'],color='k',s=100)
plt.vlines(110,400,500)
plt.text(0,450,'100 um')
plt.hlines(400,110,210)
plt.text(110,350,'100 um')
plt.xticks([])
plt.yticks([])
cbar = plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap))
#cbar.ax.set_ylabel('log(elec_distance)', rotation=270)

#ax2.ax.set_title('log(elec_distance)',rotation=270)
plt.box(False)
plt.savefig('Elec_distance.svg')

In [ ]:
outliers = df_gb[df_gb.na_lower_bound>1000]['segmentID'].values

In [ ]:
plt.figure(figsize=(4,10))
for i in exc_syns[exc_syns.Type=='apic']['Sec ID'].unique():
    plt.plot(exc_syns[(exc_syns.Type=='apic')&(exc_syns['Sec ID']==i)]['Coord X'],
             exc_syns[(exc_syns.Type=='apic')&(exc_syns['Sec ID']==i)]['Coord Y'],
             color='k',
             linewidth = 2*exc_syns[(exc_syns.Type=='apic')&(exc_syns['Sec ID']==i)]['Section_diam'].unique())
    
for i in exc_syns[exc_syns.Type=='dend']['Sec ID'].unique():
    plt.plot(exc_syns[(exc_syns.Type=='dend')&(exc_syns['Sec ID']==i)]['Coord X'],
         exc_syns[(exc_syns.Type=='dend')&(exc_syns['Sec ID']==i)]['Coord Y'],
             color='k',
             linewidth = 2*exc_syns[(exc_syns.Type=='dend')&(exc_syns['Sec ID']==i)]['Section_diam'].unique())
    

plt.scatter(exc_syns[(exc_syns.Type=='soma')&(exc_syns['Sec ID']==0)].loc[2523,'Coord X'],
         exc_syns[(exc_syns.Type=='soma')&(exc_syns['Sec ID']==0)].loc[2523,'Coord Y'],color='k',s=100)
plt.savefig('morphology.svg')

In [ ]:
inh_syns_plot = inh_syns.copy()
inh_syns_plot.loc[inh_syns.num_syns==0,'num_syns'] = 1e-1

color_field = 'num_syns'

ax = plt.figure(figsize=(4,4))
import matplotlib
from matplotlib.colors import Normalize
cmap = matplotlib.cm.get_cmap('jet')
norm = matplotlib.colors.Normalize(vmin = 0, vmax = 0.8)

for i in inh_syns_plot[inh_syns_plot.Type=='apic']['Sec ID'].unique():
    section_L = inh_syns_plot[(inh_syns_plot.Type=='apic')&(inh_syns_plot['Sec ID']==i)]['Section_L'].unique()[0]
    plt.plot(inh_syns_plot[(inh_syns_plot.Type=='apic')&(inh_syns_plot['Sec ID']==i)]['Coord X'],
             inh_syns_plot[(inh_syns_plot.Type=='apic')&(inh_syns_plot['Sec ID']==i)]['Coord Y'],
             color=cmap(norm(inh_syns_plot[(inh_syns_plot.Type=='apic')&(inh_syns_plot['Sec ID']==i)][color_field].sum()/section_L)),
             linewidth = 1*inh_syns_plot[(inh_syns_plot.Type=='apic')&(inh_syns_plot['Sec ID']==i)]['Section_diam'].unique())
    
for i in inh_syns_plot[inh_syns_plot.Type=='dend']['Sec ID'].unique():
    section_L = inh_syns_plot[(inh_syns_plot.Type=='dend')&(inh_syns_plot['Sec ID']==i)]['Section_L'].unique()[0]
    plt.plot(inh_syns_plot[(inh_syns_plot.Type=='dend')&(inh_syns_plot['Sec ID']==i)]['Coord X'],
             inh_syns_plot[(inh_syns_plot.Type=='dend')&(inh_syns_plot['Sec ID']==i)]['Coord Y'],
             color=cmap(norm(inh_syns_plot[(inh_syns_plot.Type=='dend')&(inh_syns_plot['Sec ID']==i)][color_field].sum()/section_L)),
             linewidth = 1*inh_syns_plot[(inh_syns_plot.Type=='dend')&(inh_syns_plot['Sec ID']==i)]['Section_diam'].unique())
    
for i in inh_syns_plot[inh_syns_plot.Type=='soma']['Sec ID'].unique():
    section_L = inh_syns_plot[(inh_syns_plot.Type=='soma')&(inh_syns_plot['Sec ID']==i)]['Section_L'].unique()[0]
    plt.plot(inh_syns_plot[(inh_syns_plot.Type=='soma')&(inh_syns_plot['Sec ID']==i)]['Coord X'],
             inh_syns_plot[(inh_syns_plot.Type=='soma')&(inh_syns_plot['Sec ID']==i)]['Coord Y'],
             color=cmap(norm(inh_syns_plot[(inh_syns_plot.Type=='soma')&(inh_syns_plot['Sec ID']==i)][color_field].sum()/section_L)),
             linewidth = 1*inh_syns_plot[(inh_syns_plot.Type=='soma')&(inh_syns_plot['Sec ID']==i)]['Section_diam'].unique())
#    
#
#plt.scatter(exc_syns[(exc_syns.Type=='soma')&(exc_syns['Sec ID']==0)].loc[2523,'Coord X'],
#         exc_syns[(exc_syns.Type=='soma')&(exc_syns['Sec ID']==0)].loc[2523,'Coord Y'],color='k',s=100)
plt.xticks([])
plt.yticks([])
cbar = plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap))
#cbar = plt.colorbar()
#cbar.ax.set_ylabel('log(num. inh. synapses)', rotation=270)

#ax2.ax.set_title('log(elec_distance)',rotation=270)
plt.box(False)
plt.ylim(-50,50)
plt.xlim(-50,50)
plt.savefig('inh_syns_zoom.svg')

In [ ]:
for i in np.arange(0,10):
    plt.plot(np.arange(-100,95,5),nmda_dend[i]/np.sum(nmda_dend[i]))

In [ ]:
df.loc[df.segmentID==936,'peak_value'].hist(bins=np.arange(0.003,0.01,0.001))

In [ ]:
color_field = 'num_syns'
minval = np.log10(exc_syns[color_field].min())

ax = plt.figure(figsize=(2,5))
import matplotlib
from matplotlib.colors import Normalize
cmap = matplotlib.cm.get_cmap('jet')
norm = matplotlib.colors.Normalize(vmin = 0, vmax = 8)

for i in exc_syns[exc_syns.Type=='apic']['Sec ID'].unique():
    section_L = exc_syns[(exc_syns.Type=='apic')&(exc_syns['Sec ID']==i)]['Section_L'].unique()[0]
    plt.plot(exc_syns[(exc_syns.Type=='apic')&(exc_syns['Sec ID']==i)]['Coord X'],
             exc_syns[(exc_syns.Type=='apic')&(exc_syns['Sec ID']==i)]['Coord Y'],
             color=cmap(norm(exc_syns[(exc_syns.Type=='apic')&(exc_syns['Sec ID']==i)][color_field].sum()/section_L)),
             linewidth = 1*exc_syns[(exc_syns.Type=='apic')&(exc_syns['Sec ID']==i)]['Section_diam'].unique())
    
for i in exc_syns[exc_syns.Type=='dend']['Sec ID'].unique():
    section_L = exc_syns[(exc_syns.Type=='dend')&(exc_syns['Sec ID']==i)]['Section_L'].unique()[0]
    plt.plot(exc_syns[(exc_syns.Type=='dend')&(exc_syns['Sec ID']==i)]['Coord X'],
             exc_syns[(exc_syns.Type=='dend')&(exc_syns['Sec ID']==i)]['Coord Y'],
             color=cmap(norm(exc_syns[(exc_syns.Type=='dend')&(exc_syns['Sec ID']==i)][color_field].sum()/section_L)),
             linewidth = 1*exc_syns[(exc_syns.Type=='dend')&(exc_syns['Sec ID']==i)]['Section_diam'].unique())
#    
#
plt.scatter(exc_syns[(exc_syns.Type=='soma')&(exc_syns['Sec ID']==0)].loc[2523,'Coord X'],
         exc_syns[(exc_syns.Type=='soma')&(exc_syns['Sec ID']==0)].loc[2523,'Coord Y'],color='k',s=100)
plt.vlines(110,400,500)
plt.text(0,450,'100 um')
plt.hlines(400,110,210)
plt.text(110,350,'100 um')
plt.xticks([])
plt.yticks([])
cbar = plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap))
#cbar.ax.set_ylabel('log(elec_distance)', rotation=270)

#ax2.ax.set_title('log(elec_distance)',rotation=270)
plt.box(False)
plt.savefig('exc_syns.svg')

In [ ]:
segs = pd.read_csv('/Volumes/TOSHIBA EXT/L5NeuronSimulation_new/L5NeuronSimulation/MorphAnalysis/Segments.csv')

In [ ]:
segs['median_v'] = 0

In [ ]:
for s in segs.index:
    print(s)
    segs.loc[s,'median_v'] = np.median(v['report']['biophysical']['data'][1000:10000,s])

In [ ]:
segs.head()

In [ ]:
color_field = 'median_v'

ax = plt.figure(figsize=(2,5))
import matplotlib
from matplotlib.colors import Normalize
cmap = matplotlib.cm.get_cmap('jet')
norm = matplotlib.colors.Normalize(vmin = -80, vmax = 0)

for i in segs[segs.Type=='apic']['Sec ID'].unique():
    section_L = segs[(segs.Type=='apic')&(segs['Sec ID']==i)]['Section_L'].unique()[0]
    plt.plot(segs[(segs.Type=='apic')&(segs['Sec ID']==i)]['Coord X'],
             segs[(segs.Type=='apic')&(segs['Sec ID']==i)]['Coord Y'],
             color=cmap(norm(segs[(segs.Type=='apic')&(segs['Sec ID']==i)][color_field].mean())),
             linewidth = 1*segs[(segs.Type=='apic')&(segs['Sec ID']==i)]['Section_diam'].unique())
    
for i in segs[segs.Type=='dend']['Sec ID'].unique():
    section_L = segs[(segs.Type=='dend')&(segs['Sec ID']==i)]['Section_L'].unique()[0]
    plt.plot(segs[(segs.Type=='dend')&(segs['Sec ID']==i)]['Coord X'],
             segs[(segs.Type=='dend')&(segs['Sec ID']==i)]['Coord Y'],
             color=cmap(norm(segs[(segs.Type=='dend')&(segs['Sec ID']==i)][color_field].mean())),
             linewidth = 1*segs[(segs.Type=='dend')&(segs['Sec ID']==i)]['Section_diam'].unique())
#    
#
plt.scatter(exc_syns[(exc_syns.Type=='soma')&(exc_syns['Sec ID']==0)].loc[2523,'Coord X'],
         exc_syns[(exc_syns.Type=='soma')&(exc_syns['Sec ID']==0)].loc[2523,'Coord Y'],color='k',s=100)
plt.vlines(110,400,500)
plt.text(0,450,'100 um')
plt.hlines(400,110,210)
plt.text(110,350,'100 um')
plt.xticks([])
plt.yticks([])
cbar = plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap))
#cbar.ax.set_ylabel('log(elec_distance)', rotation=270)

#ax2.ax.set_title('log(elec_distance)',rotation=270)
plt.box(False)
plt.savefig('median_v.svg')

In [ ]:
exc_syns[(exc_syns['Coord Y']>500) & (exc_syns['Coord Y']<700)]['Sec ID'].unique()


ca_df = pd.read_csv('ca_df.csv') # also this

In [ ]:
p = np.random.choice(ca_df[(~pd.isnull(ca_df.ca_lower_bound))].index)
ca_df.loc[p,'segmentID'], ca_df.loc[p,'ca_lower_bound']

In [ ]:
plt.plot(exc_syns[exc_syns.Type=='apic']['Coord X'],
         exc_syns[exc_syns.Type=='apic']['Coord Y'],'.')
plt.plot(exc_syns.loc[(exc_syns.Type=='apic')&(exc_syns['Section_diam']>1.0)]['Coord X'],
         exc_syns.loc[(exc_syns.Type=='apic')&(exc_syns['Section_diam']>1.0)]['Coord Y'],'r.')
plt.plot(exc_syns.loc[(exc_syns.Type=='apic')&(exc_syns['Sec ID']==50)]['Coord X'],
         exc_syns.loc[(exc_syns.Type=='apic')&(exc_syns['Sec ID']==50)]['Coord Y'],'m.')

         df_noout = df[(df.segmentID.isin(outliers))&(df.peak_value>0.006)]

In [ ]:
t = 1090075
s = 1312

plt.plot(np.arange(-60,80,.1),hva['report']['biophysical']['data'][t-600:t+800,s],color='r',label='hva')
plt.plot(np.arange(-60,80,.1),lva['report']['biophysical']['data'][t-600:t+800,s],color='c',label='lva')
plt.legend(loc='center left')
plt.vlines(-20,-0.001,-0.0015,color='r')
plt.text(-18,-0.00125,'0.5 pA')
plt.ylabel('current (nA)')

plt.twinx()

plt.plot(np.arange(-60,80,.1),v['report']['biophysical']['data'][t-600:t+800,s],color='k')
plt.plot(np.arange(-60,80,.1),v['report']['biophysical']['data'][t-600:t+800,0],color='k',alpha=0.2)
plt.hlines(-50,70,80)
plt.text(65,-55,'10 ms')
plt.vlines(70,-40,-50)
plt.text(50,-45,'10 mV')
plt.ylabel('voltage (mV)')
plt.title('Ca2+ spike')
plt.text(60,-65,'soma',alpha=0.2)
plt.savefig('Caspike.svg')

In [ ]:
na_df_new = pd.concat((df_noout[['segmentID','na_lower_bound','peak_value']],
                       na_df.loc[~na_df.segmentID.isin(outliers),['segmentID','na_lower_bound','peak_value']]))

In [ ]:
na_df_new.to_csv('na_df.csv',index=False)

inh_syns_plot = inh_syns.copy()
inh_syns_plot.loc[inh_syns.num_syns==0,'num_syns'] = 1e-1

color_field = 'num_syns'

ax = plt.figure(figsize=(2,5))
import matplotlib
from matplotlib.colors import Normalize
cmap = matplotlib.cm.get_cmap('jet')
norm = matplotlib.colors.Normalize(vmin = 0, vmax = 5)

for i in inh_syns_plot[inh_syns_plot.Type=='apic']['Sec ID'].unique():
    section = inh_syns_plot[(inh_syns_plot.Type=='apic')&(inh_syns_plot['Sec ID']==i)]
    for j in section.index.tolist()[:-1]:
        plt.plot(section.loc[j:j+1,'Coord X'],
                 section.loc[j:j+1,'Coord Y'],
             color=cmap(norm(section.loc[j:j+1,color_field].values[0])),
             linewidth = 1*section.loc[j:j+1,'Section_diam'].unique())
    
for i in inh_syns_plot[inh_syns_plot.Type=='dend']['Sec ID'].unique():
    section = inh_syns_plot[(inh_syns_plot.Type=='dend')&(inh_syns_plot['Sec ID']==i)]
    for j in section.index.tolist()[:-1]:
        plt.plot(section.loc[j:j+1,'Coord X'],
                 section.loc[j:j+1,'Coord Y'],
             color=cmap(norm(section.loc[j:j+1,color_field].values[0])),
             linewidth = 1*section.loc[j:j+1,'Section_diam'].unique())
        
plt.scatter(inh_syns_plot[(inh_syns_plot.Type=='soma')&(inh_syns_plot['Sec ID']==0)].loc[2523,'Coord X'],
         inh_syns_plot[(inh_syns_plot.Type=='soma')&(inh_syns_plot['Sec ID']==0)].loc[2523,'Coord Y'],color='k',s=100)
plt.vlines(110,400,500)
plt.text(0,450,'100 um')
plt.hlines(400,110,210)
plt.text(110,350,'100 um')
plt.xticks([])
plt.yticks([])
cbar = plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap))
#cbar.ax.set_ylabel('log(elec_distance)', rotation=270)

#ax2.ax.set_title('log(elec_distance)',rotation=270)
plt.box(False)
plt.savefig('inh_syns.svg')

In [ ]:
na_df = pd.read_csv('na_df.csv')

In [ ]:
segs_na_df = segs.set_index('segmentID').join(na_df.set_index('segmentID')).reset_index()

plt.figure(figsize=(10,4))
s1 = 559133+1000#np.random.choice(range(v['report']['biophysical']['data'].shape[0]))
plt.plot(np.arange(0,400,.1),v['report']['biophysical']['data'][s1:s1+4000,0],color='k')
#1433413, 1161455
plt.hlines(-10,40,70)
plt.text(40,-15,'30 ms')
plt.vlines(40,-10,0)
plt.text(5,-5,'10 mV')
plt.box(False)
plt.xticks([])
plt.yticks([])

In [ ]:
def get_sta(segs_na_df, spktimes):
    na_dend = np.zeros((10,39,))
    na_apic = np.zeros((10,39,))
    
    edges_dend=np.quantile(segs_na_df[(~pd.isnull(segs_na_df.na_lower_bound))&
                                       (segs_na_df.Type=='dend')]['Elec_distance'].unique(),np.arange(0,1.1,0.1))
    
    edges_apic=np.quantile(segs_na_df[(~pd.isnull(segs_na_df.na_lower_bound))&
                                       (segs_na_df.Type=='apic')]['Elec_distance'].unique(),np.arange(0,1.1,0.1))
    
    actual_spktimes = []
    c = 0
    for i in np.sort(spktimes):
        # excludes bursts
        if i-c > 10:
            for e in np.arange(0,10):
                na_inds = segs_na_df[(~pd.isnull(segs_na_df.na_lower_bound))&
                                     (segs_na_df.Type=='dend')&
                                     (segs_na_df.Elec_distance>edges_dend[e])&
                                     (segs_na_df.Elec_distance<=edges_dend[e+1])]['na_lower_bound'].values.astype(int)
    
                x2, _ = np.histogram(na_inds/10,bins=np.arange(np.floor(i)-20,np.floor(i)+20,1))
                na_dend[e] += x2
                
                na_inds = segs_na_df[(~pd.isnull(segs_na_df.na_lower_bound))&
                                     (segs_na_df.Type=='apic')&
                                     (segs_na_df.Elec_distance>edges_apic[e])&
                                     (segs_na_df.Elec_distance<=edges_apic[e+1])]['na_lower_bound'].values.astype(int)
    
                x2, _ = np.histogram(na_inds/10,bins=np.arange(np.floor(i)-20,np.floor(i)+20,1))
                na_apic[e] += x2
            
            actual_spktimes.append(i)
        c = i
    return na_dend, na_apic

In [ ]:
na_dend, na_apic = get_sta(segs_na_df, spktimes)
rand_spktimes = np.sort(np.random.choice(np.arange(0,150000), spktimes.shape[0]))
na_dend_rand, na_apic_rand = get_sta(segs_na_df, rand_spktimes)

In [ ]:
ca_df.reset_index(inplace=True)

In [ ]:
x = ca_df[~pd.isnull(ca_df.ca_lower_bound)].reset_index()
i = np.random.choice(x.index)

In [ ]:
plt.figure(figsize=(10,4))
s1 = 1147000#np.random.choice(range(v['report']['biophysical']['data'].shape[0]))
t = 1500
plt.plot(np.arange(0,400,.1),v['report']['biophysical']['data'][s1:s1+4000,t],color='g')
na_spks = na_df[(na_df.segmentID==t)&(na_df.na_lower_bound>s1) & (na_df.na_lower_bound<s1+4000)]
plt.plot(np.arange(0,400,.1)[na_spks['na_lower_bound'].astype(int).values-s1],
         v['report']['biophysical']['data'][na_spks['na_lower_bound'].astype(int).values,t],'r*')
nmda_spks = nmda_df[(nmda_df.segmentID==t)&(nmda_df.nmda_lower_bound>s1) & (nmda_df.nmda_lower_bound<s1+4000)]
plt.plot(np.arange(0,400,.1)[nmda_spks['nmda_lower_bound'].astype(int).values-s1],
         v['report']['biophysical']['data'][nmda_spks['nmda_lower_bound'].astype(int).values,t],'b*')
ca_spks = ca_df[(ca_df.segmentID==t)&(ca_df.ca_lower_bound>s1) & (ca_df.ca_lower_bound<s1+4000)]
plt.plot(np.arange(0,400,.1)[ca_spks['ca_lower_bound'].astype(int).values-s1],
         v['report']['biophysical']['data'][ca_spks['ca_lower_bound'].astype(int).values,t],'m*')
plt.hlines(-60,0,400,color='g',linestyle='dashed',alpha=0.2)
plt.text(10,-57,'-60 mV')
#1433413, 1161455
plt.hlines(-50,70,100)
plt.text(70,-55,'30 ms')
plt.vlines(70,-50,-40)
plt.text(40,-45,'10 mV')
plt.box(False)
plt.xticks([])
plt.yticks([])
plt.savefig('fig2C.svg')

In [ ]:
plt.figure(figsize=(10,4))
s1 = 1147000#np.random.choice(range(v['report']['biophysical']['data'].shape[0]))
plt.plot(np.arange(0,400,.1),v['report']['biophysical']['data'][s1:s1+4000,0],color='k')
plt.hlines(-60,0,400,color='k',linestyle='dashed',alpha=0.2)
plt.text(10,-57,'-60 mV')
#1433413, 1161455
plt.hlines(-50,70,100)
plt.text(70,-55,'30 ms')
plt.vlines(70,-50,-40)
plt.text(40,-45,'10 mV')
plt.box(False)
plt.xticks([])
plt.yticks([])
plt.savefig('fig2E.svg')

In [ ]:
plt.figure(figsize=(10,4))
s1 = 1147000#np.random.choice(range(v['report']['biophysical']['data'].shape[0]))
t = 1046
plt.plot(np.arange(0,400,.1),v['report']['biophysical']['data'][s1:s1+4000,t],color='m')
na_spks = na_df[(na_df.segmentID==t)&(na_df.na_lower_bound>s1) & (na_df.na_lower_bound<s1+4000)]
plt.plot(np.arange(0,400,.1)[na_spks['na_lower_bound'].astype(int).values-s1],
         v['report']['biophysical']['data'][na_spks['na_lower_bound'].astype(int).values,t],'r*')
nmda_spks = nmda_df[(nmda_df.segmentID==t)&(nmda_df.nmda_lower_bound>s1) & (nmda_df.nmda_lower_bound<s1+4000)]
plt.plot(np.arange(0,400,.1)[nmda_spks['nmda_lower_bound'].astype(int).values-s1],
         v['report']['biophysical']['data'][nmda_spks['nmda_lower_bound'].astype(int).values,t],'b*')
plt.hlines(-60,0,400,color='m',linestyle='dashed',alpha=0.2)
plt.text(10,-57,'-60 mV')
#1433413, 1161455
plt.hlines(-50,70,100)
plt.text(70,-55,'30 ms')
plt.vlines(70,-50,-40)
plt.text(40,-45,'10 mV')
plt.box(False)
plt.xticks([])
plt.yticks([])
plt.savefig('fig2D.svg')

In [ ]:
plt.figure(figsize=(10,4))
s1 = 1147000#np.random.choice(range(v['report']['biophysical']['data'].shape[0]))
t = 1249
plt.plot(np.arange(0,400,.1),v['report']['biophysical']['data'][s1:s1+4000,t],color='c')
na_spks = na_df[(na_df.segmentID==t)&(na_df.na_lower_bound>s1) & (na_df.na_lower_bound<s1+4000)]
plt.plot(np.arange(0,400,.1)[na_spks['na_lower_bound'].astype(int).values-s1],
         v['report']['biophysical']['data'][na_spks['na_lower_bound'].astype(int).values,t],'r*')
nmda_spks = nmda_df[(nmda_df.segmentID==t)&(nmda_df.nmda_lower_bound>s1) & (nmda_df.nmda_lower_bound<s1+4000)]
plt.plot(np.arange(0,400,.1)[nmda_spks['nmda_lower_bound'].astype(int).values-s1],
         v['report']['biophysical']['data'][nmda_spks['nmda_lower_bound'].astype(int).values,t],'b*')
plt.hlines(-60,0,400,color='c',linestyle='dashed',alpha=0.2)
plt.text(10,-57,'-60 mV')
#1433413, 1161455
plt.hlines(-50,70,100)
plt.text(70,-55,'30 ms')
plt.vlines(70,-50,-40)
plt.text(40,-45,'10 mV')
plt.box(False)
plt.xticks([])
plt.yticks([])
plt.savefig('fig2B.svg')

In [ ]:
for i in np.random.choice(ca_df[(ca_df.ca_lower_bound>2000) & (ca_df.ca_lower_bound<1400000)].index,100):
    seg = ca_df.loc[i,'segmentID']
    if not pd.isnull(ca_df.loc[i,'ca_lower_bound']):
        spkt = int(ca_df.loc[i,'ca_lower_bound'])
        
        trace = -(hva['report']['biophysical']['data'][spkt-1000:spkt+1000,seg] + \
                lva['report']['biophysical']['data'][spkt-1000:spkt+1000,seg] + \
                ih['report']['biophysical']['data'][spkt-1000:spkt+1000,seg])
        peak_value = np.max(trace)
        half_peak = peak_value/2
        duration = np.arange(0,2000)[trace>half_peak] + spkt - 10
        ca_df.loc[i,'duration_low'] = duration[0]
        ca_df.loc[i,'duration_high'] = duration[-1]
        ca_df.loc[i,'peak_value'] = peak_value
    else:
        ca_df.loc[i,'duration_low'] = np.nan
        ca_df.loc[i,'duration_high'] = np.nan
        ca_df.loc[i,'peak_value'] = np.nan
###########################################
plt.figure(figsize=(10,4))
s1 = 1147000#np.random.choice(range(v['report']['biophysical']['data'].shape[0]))
t = 1800
plt.plot(np.arange(0,400,.1),v['report']['biophysical']['data'][s1:s1+4000,t],color='r')
na_spks = na_df[(na_df.segmentID==t)&(na_df.na_lower_bound>s1) & (na_df.na_lower_bound<s1+4000)]
plt.plot(np.arange(0,400,.1)[na_spks['na_lower_bound'].astype(int).values-s1],
         v['report']['biophysical']['data'][na_spks['na_lower_bound'].astype(int).values,t],'r*')
nmda_spks = nmda_df[(nmda_df.segmentID==t)&(nmda_df.nmda_lower_bound>s1) & (nmda_df.nmda_lower_bound<s1+4000)]
plt.plot(np.arange(0,400,.1)[nmda_spks['nmda_lower_bound'].astype(int).values-s1],
         v['report']['biophysical']['data'][nmda_spks['nmda_lower_bound'].astype(int).values,t],'b*')
plt.hlines(-60,0,400,color='r',linestyle='dashed',alpha=0.2)
plt.text(0,-57,'-60 mV')
#1433413, 1161455
plt.hlines(-60,170,200)
plt.text(170,-65,'30 ms')
plt.vlines(170,-60,-50)
plt.text(140,-55,'10 mV')
plt.box(False)
plt.xticks([])
plt.yticks([])
plt.savefig('fig2A.svg')

In [ ]:
plt.figure(figsize=(10,4))
s1 = 1147000#np.random.choice(range(v['report']['biophysical']['data'].shape[0]))
t = 88

plt.plot(np.arange(0,400,.1),v['report']['biophysical']['data'][s1:s1+4000,t],color='orange')
na_spks = na_df[(na_df.segmentID==t)&(na_df.na_lower_bound>s1) & (na_df.na_lower_bound<s1+4000)]
plt.plot(np.arange(0,400,.1)[na_spks['na_lower_bound'].astype(int).values-s1],
         v['report']['biophysical']['data'][na_spks['na_lower_bound'].astype(int).values,t],'r*')
nmda_spks = nmda_df[(nmda_df.segmentID==t)&(nmda_df.nmda_lower_bound>s1) & (nmda_df.nmda_lower_bound<s1+4000)]
plt.plot(np.arange(0,400,.1)[nmda_spks['nmda_lower_bound'].astype(int).values-s1],
         v['report']['biophysical']['data'][nmda_spks['nmda_lower_bound'].astype(int).values,t],'b*')
plt.hlines(-60,0,400,color='orange',linestyle='dashed',alpha=0.2)
plt.text(10,-57,'-60 mV')
#1433413, 1161455
plt.hlines(-50,70,100)
plt.text(70,-55,'30 ms')
plt.vlines(70,-50,-40)
plt.text(40,-45,'10 mV')
plt.box(False)
plt.xticks([])
plt.yticks([])

plt.savefig('fig2F.svg')

In [ ]:
ca_df[~pd.isnull(ca_df.duration_high)]
###
t = 482

plt.figure(figsize=(10,4))
s1 = 1147000#np.random.choice(range(v['report']['biophysical']['data'].shape[0]))
plt.plot(np.arange(0,400,.1),v['report']['biophysical']['data'][s1:s1+4000,t],color='grey')
na_spks = na_df[(na_df.segmentID==t)&(na_df.na_lower_bound>s1) & (na_df.na_lower_bound<s1+4000)]
plt.plot(np.arange(0,400,.1)[na_spks['na_lower_bound'].astype(int).values-s1],
         v['report']['biophysical']['data'][na_spks['na_lower_bound'].astype(int).values,t],'r*')
nmda_spks = nmda_df[(nmda_df.segmentID==t)&(nmda_df.nmda_lower_bound>s1) & (nmda_df.nmda_lower_bound<s1+4000)]
plt.plot(np.arange(0,400,.1)[nmda_spks['nmda_lower_bound'].astype(int).values-s1],
         v['report']['biophysical']['data'][nmda_spks['nmda_lower_bound'].astype(int).values,t],'b*')
plt.hlines(-60,0,400,color='grey',linestyle='dashed',alpha=0.2)
plt.text(10,-57,'-60 mV')
#1433413, 1161455
plt.hlines(-50,70,100)
plt.text(70,-55,'30 ms')
plt.vlines(70,-50,-40)
plt.text(40,-45,'10 mV')
plt.box(False)
plt.xticks([])
plt.yticks([])
plt.savefig('fig2G.svg')

In [ ]:
segs[segs.Type=='soma']

In [ ]:
seg = 2403
lower_bound = 204744
#upper_bound = 
plt.plot(-(hva['report']['biophysical']['data'][lower_bound-1000:lower_bound+1000,seg]+\
         lva['report']['biophysical']['data'][lower_bound-1000:lower_bound+1000,seg]+\
         ih['report']['biophysical']['data'][lower_bound-1000:lower_bound+1000,seg]))
plt.plot(1000,-(hva['report']['biophysical']['data'][lower_bound,seg]+\
         lva['report']['biophysical']['data'][lower_bound,seg]+\
         ih['report']['biophysical']['data'][lower_bound,seg]),'*')

plt.plot(1000+205494-lower_bound,-(hva['report']['biophysical']['data'][205494,seg]+\
         lva['report']['biophysical']['data'][205494,seg]+\
         ih['report']['biophysical']['data'][205494,seg]),'*')

plt.twinx()
plt.plot(v['report']['biophysical']['data'][lower_bound-1000:lower_bound+1000,seg],color='k')